<a href="https://colab.research.google.com/github/adam1brownell/cityguide_llm/blob/main/la_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install langchain
!pip3 install langchain_pinecone
!pip3 install langchain[docarray]
!pip3 install docarray
!pip3 install pypdf

In [2]:
!pip install langchain_community
!curl https://ollama.ai/install.sh | sh
!ollama pull llama2
!pip install selenium

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10975    0 10975    0     0  29147      0 --:--:-- --:--:-- --:--:-- 29188
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Error: could not connect to ollama app, is it running?


## Load our Model

In [3]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

## Load in our model
MODEL = "llama2"

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

## Load (or Build) Documents

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run headless Chrome, optional
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

Pull Links (Discover LA) from Scratch

In [5]:
# Navigate to the main page
driver = webdriver.Chrome(options=chrome_options)

url = "https://www.discoverlosangeles.com/"
driver.get(url)
links = []

# Get all URLs on the homepage
def get_all_links(links, driver):
    elements = driver.find_elements(By.TAG_NAME, 'a')
    for elem in elements:
        href = elem.get_attribute('href')
        if href and href.startswith('https://www.discoverlosangeles.com'):
            if href not in links:
                links.append(href)
    return(links)

all_links = get_all_links(links, driver)
print(f"Found {len(all_links)} links on the homepage.")
driver.quit()

Found 92 links on the homepage.


Crawl homepage links for more links

In [ ]:
# Brute Force Crawler
driver = webdriver.Chrome(options=chrome_options)
i = 0
# while i < len(all_links):
for i in range(100):
    url = all_links[i]
    driver.get(url)
    all_links = get_all_links(all_links, driver)
    i += 1

    if i % 25 == 0:
        print(f"{100*i/len(all_links):.2f}% of total links explored")

## pull text
driver = webdriver.Chrome(options=chrome_options)
documents = []
c = 0
STOP_AT = len(all_links)
for url in all_links:
    if 'node/' in url:
        continue
    ## Can upgrade to hotels, food and drink soon. This just makes things quicker
    if ('event/' not in url) and ('things-to-do/' not in url):
        continue
    # print(url)
    driver.get(url)
    article = driver.find_element(By.XPATH, '//div[contains(@class, "page-content")]')
    if article:
        text = article.text
        documents.append(text)
    c += 1
    if c > STOP_AT:
        break

# Close the browse
driver.quit()

3.97% of total links explored
3.64% of total links explored
4.75% of total links explored
4.24% of total links explored
https://www.discoverlosangeles.com/things-to-do/attractions-tours
https://www.discoverlosangeles.com/things-to-do/arts
https://www.discoverlosangeles.com/things-to-do/wellness
https://www.discoverlosangeles.com/things-to-do/shopping
https://www.discoverlosangeles.com/things-to-do/aapi
https://www.discoverlosangeles.com/things-to-do/discover-barbie-locations-in-los-angeles
https://www.discoverlosangeles.com/things-to-do/discover-dragons-in-los-angeles
https://www.discoverlosangeles.com/things-to-do/hiking-in-los-angeles-las-best-trails
https://www.discoverlosangeles.com/things-to-do/the-best-beaches-in-los-angeles
https://www.discoverlosangeles.com/things-to-do/discover-the-hollywood-vinyl-district
https://www.discoverlosangeles.com/things-to-do/nightlife
https://www.discoverlosangeles.com/event/2024/07/27/national-day-of-the-cowboy-and-cowgirl
https://www.discoverlosa

## Load Documents

In [ ]:
print(True)

## Build Vector Store

In [ ]:
## Build Vector Store
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import DocArrayInMemorySearch

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
text_documents = text_splitter.split_documents(documents)
vectorstore = DocArrayInMemorySearch.from_documents(text_documents, embedding=embeddings)
retriever = vectorstore.as_retriever()

## Set up Q/A system

In [ ]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't
answer the question, answer with "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

In [ ]:
retrieved_context = retriever.invoke("laptop")
questions = [
    "What are some fun things to do in Los Angeles this weekend?",
    "What are some interesting paid events that are happening in Los Angeles soon? How much are they?"
]

for question in questions:
    retrieved_context = retriever.invoke(question)
    formatted_prompt = prompt.format(context=retrieved_context, question=question)
    response_from_model = model.invoke(formatted_prompt)
    parsed_response = parser.parse(response_from_model)

    print(f"Question: {question}")
    print(f"Answer: {parsed_response}")
    print()